In [ ]:
import sys
sys.path.append('../../Common/')

import CommonMT5

In [ ]:
# Định nghĩa hàm tín hiệu mua
def buy_signal(row):
    if row['Close'] > row['candle_avg'] and row['macd_histogram'] > row['macd_histogram_shifted']:
        return True
    else:
        return False

# Định nghĩa hàm tín hiệu bán
def sell_signal(row):
    if row['Close'] < row['candle_avg'] and row['macd_histogram'] < row['macd_histogram_shifted']:
        return True
    else:
        return False

def detectSignal(symbol, from_date, to_date, timeframe):

    import pandas as pd
    import plotly.graph_objects as go
    import redis
    import numpy as np
    import ta
    from datetime import datetime

    # ##############################################Step 1: Lấy dữ liệu##############################################
    data = CommonMT5.CommonMT5.loaddataMT5_FromTo(symbol, from_date, to_date, timeframe)
    
    # ##############################################Step 2: Chiến lược##############################################  
    # Tính toán các chỉ báo kỹ thuật
    
    # Tính toán các đường EMA (Exponential Moving Average) cho MACD
    ema_short = data['Close'].ewm(span=12, adjust=False).mean()  # Thường là EMA 12 ngày
    ema_long = data['Close'].ewm(span=26, adjust=False).mean()   # Thường là EMA 26 ngày

    # Tính toán đường MACD và đường Signal
    macd = ema_short - ema_long
    signal = macd.ewm(span=9, adjust=False).mean()  # Thường là EMA 9 ngày của đường MACD

    # Tính toán histogram MACD (chênh lệch giữa MACD và đường Signal)
    data['macd_histogram'] = macd - signal

    # Tạo cột 'candle_avg' dựa trên điều kiện đã định nghĩa
    data['candle_avg'] = data['Close'].rolling(window=2).mean()

    # Tạo một cột 'macd_histogram_shifted' bằng cách dịch chuyển 'macd_histogram' xuống một hàng
    data['macd_histogram_shifted'] = data['macd_histogram'].shift(1)

    # Áp dụng hàm tín hiệu mua và bán
    data['Buy_Signal'] = data.apply(lambda row: buy_signal(row), axis=1)
    data['Sell_Signal'] = data.apply(lambda row: sell_signal(row), axis=1)

    
    # ##############################################Step 3: Đẩy dữ liệu qua Redis##############################################
    # Nếu có tín hiệu thì đẩy qua Redis
    # Datetime  Open    High    Low	Close   Volume  SMA short_ema   long_ema    MACD    Signal_Line Buy_Signal  Sell_Signal
    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)
    # Tạo hash key
    hash_key = 'OG_HISTOGRAM'
    last_record = data.iloc[-1]
   
    # Chuyển đổi record cuối cùng thành từ điển và lưu vào Redis dưới dạng hash
    if(last_record['Buy_Signal'] == True or last_record['Sell_Signal'] == True):
        for field, value in last_record.to_dict().items():
            # Chuyển đổi giá trị uint64 và Timestamp thành chuỗi
            if isinstance(value, pd.Timestamp):
                value = value.isoformat()
            elif isinstance(value, (int, np.uint64)):
                value = str(value)
            r.hset(hash_key, field, value)  
            r.hset(hash_key, 'Symbol', symbol)
            r.hset(hash_key, 'Insertdate', datetime.now().isoformat())
        print(last_record)   
    else:
        print('Không có tín hiệu!')
    # ##############################################Step 4: Vẽ biểu đồ##############################################  
    # data.set_index('Datetime', inplace=True)
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ giá đóng cửa và SMA
    # fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Giá Đóng Cửa', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='SMA', line=dict(color='orange')))
    # # Vẽ các điểm cho tín hiệu mua
    # buy_signals = data[data['Buy_Signal']]
    # fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', size=10, color='green')))
    # # Vẽ các điểm cho tín hiệu bán
    # sell_signals = data[data['Sell_Signal']]
    # fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', size=10, color='red')))
    # fig.update_layout(title='Giá Đóng Cửa và SMA', yaxis_title='Giá', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

    # ####################################################################################################
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ MACD và Signal Line
    # fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['Signal_Line'], mode='lines', name='Signal Line', line=dict(color='orange')))
    # fig.update_layout(title='MACD và Signal Line', yaxis_title='MACD', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

In [ ]:
# import MetaTrader5 as mt5
# from datetime import datetime, timedelta
# import ta
# import schedule
# import time

# def schedule_detectSignal(timeframe):
#     symbol = 'EURUSD'
#     from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
#     to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
#     # timeframe = mt5.TIMEFRAME_M30 # 1m
#     # 0x4000
#     detectSignal(symbol, from_date, to_date, timeframe)

# # Danh sách các phút cụ thể bạn muốn hàm được chạy
# run_minutes = list(range(0,60,30))
# while True:
#     current_time = datetime.now()
#     current_minute = current_time.minute
#     # Kiểm tra xem phút hiện tại có nằm trong danh sách các phút cần chạy hàm không
#     if current_minute in run_minutes:
#         # Kiểm tra xem đã chạy hàm trong phút hiện tại hay chưa
#         last_run = getattr(schedule_detectSignal, 'last_run', None)
#         if last_run is None or last_run != current_minute:
#             schedule_detectSignal(mt5.TIMEFRAME_M30)
#             # Lưu lại phút cuối cùng mà hàm đã chạy
#             setattr(schedule_detectSignal, 'last_run', current_minute)

#     # Nghỉ 1 giây trước khi kiểm tra lại
#     time.sleep(1)

In [ ]:
# import MetaTrader5 as mt5
# from datetime import datetime, timedelta
# import ta
# import schedule
# import time

# def schedule_job():
#     symbol = 'EURUSD'
#     from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')
#     to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
#     timeframe = mt5.TIMEFRAME_H1
#     detectSignal(symbol, from_date, to_date, timeframe)

# # Lên lịch chạy hàm tại các giờ cố định trong ngày
# schedule.every().day.at("00:01").do(schedule_job)
# schedule.every().day.at("01:01").do(schedule_job)
# schedule.every().day.at("02:01").do(schedule_job)
# schedule.every().day.at("04:00").do(schedule_job)
# schedule.every().day.at("08:00").do(schedule_job)
# schedule.every().day.at("12:00").do(schedule_job)
# schedule.every().day.at("16:00").do(schedule_job)
# schedule.every().day.at("20:00").do(schedule_job)

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [5]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import ta
import schedule
import time

def schedule_job():
    symbol = 'EURUSD'
    from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = mt5.TIMEFRAME_H1
    detectSignal(symbol, from_date, to_date, timeframe)

# Lên lịch chạy hàm tại các giờ cố định trong ngày
for hour in range(24):  # Loop from 0 to 23
    # Format the time string with leading zeros for hours
    time_string = "{:02d}:04".format(hour)
    schedule.every().day.at(time_string).do(schedule_job)

# # Lên lịch chạy hàm tại các phút cụ thể trong mỗi giờ
# for hour in range(24):  # Loop from 0 to 23
#     for minute in [1, 24, 31, 46]:  # Các phút cụ thể
#         # Format the time string with leading zeros for hours and minutes
#         time_string = "{:02d}:{:02d}".format(hour, minute)
#         schedule.every().day.at(time_string).do(schedule_job)

while True:
    schedule.run_pending()
    time.sleep(1)